# Using Web APIs

An API, or Application Program Interface, allows one program to *talk* to another program. Many websites or services provide an API so you can query for information in an automated way. 

For mapping and spatial analysis, being able to use APIs is critical. For the longest time, Google Maps API was the most popular API on the web. APIs allow you to query web servers and get results without downloading data or running computation on your machine. 

Common use cases for using APIs for spatial analysis are

- Getting directions / routing
- Route optimization
- Geocoding
- Downloading data
- Getting real-time weather data
- ...

The provide of such APIs have many ways to implement an API. There are standards such as REST, SOAP, GraphQL etc. *REST* is the most populat standard for web APIs, and for geospatial APIs. REST APIs are used over HTTP and thus called web APIs.


## Understanding JSON and GeoJSON

JSON stands for **J**ava**S**cript **O**bject **N**otation. It is a format for storing and transporting data, and is the de-facto standard for data exchanged by APIs. GeoJSON is an extension of the JSON format that is commonly used to represent spatial data.

Python has a built-in `json` module that has methods for reading json data and converting it to Python objects, and vice-versa. In this example, we are using the `requests` module for querying the API which conveniently does the conversion for us. But it is useful to learn the basics of working with JSON in Python.

The GeoJSON data contains *features*, where each feature has some *properties* and a *geometry*. 

In [2]:
geojson_string = '''
{
  "type": "FeatureCollection",
  "features": [
    {"type": "Feature",
      "properties": {"name": "San Francisco"},
      "geometry": {"type": "Point", "coordinates": [-121.5687, 37.7739]}
    }
  ]
}
'''
print(geojson_string)


{
  "type": "FeatureCollection",
  "features": [
    {"type": "Feature",
      "properties": {"name": "San Francisco"},
      "geometry": {"type": "Point", "coordinates": [-121.5687, 37.7739]}
    }
  ]
}



To convert a JSON string to a Python object (i.e. parsing JSON), we can use the `json.loads()` method.

In [3]:
import json

data = json.loads(geojson_string)
print(type(data))
print(data)

<class 'dict'>
{'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'properties': {'name': 'San Francisco'}, 'geometry': {'type': 'Point', 'coordinates': [-121.5687, 37.7739]}}]}


Now that we have *parsed* the GeoJSON string and have a Python object, we can extract infromation from it. The data is stored in a *FeatureCollection* - which is a list of *features*. In our example, we have just 1 feature inside the feature collection, so we can access it by using index **0**.

In [3]:
city_data = data['features'][0]
print(city_data)

{'type': 'Feature', 'properties': {'name': 'San Francisco'}, 'geometry': {'type': 'Point', 'coordinates': [-121.5687, 37.7739]}}


The feature representation is a dictionary, and individual items can be accesses using the *keys*

In [4]:
city_name = city_data['properties']['name']
city_coordinates = city_data['geometry']['coordinates']
print(city_name, city_coordinates)

San Francisco [-121.5687, 37.7739]


## The `requests` module

To query a server, we send a **GET** request with some parameters and the server sends a response back. The `requests` module allows you to send HTTP requests and parse the responses using Python. 

The response contains the data received from the server. It contains the HTTP *status_code* which tells us if the request was successful. HTTP code 200 stands for *Sucess OK*.


In [4]:
import requests

response = requests.get('https://www.spatialthoughts.com')

print(response.status_code)

200


## Calculating Distance using OpenRouteService API

![](images/python_foundation/ors_direction.png)

[OpenRouteService (ORS)](https://openrouteservice.org/) provides a free API for routing, distance matrix, geocoding, route optimization etc. using OpenStreetMap data. We will learn how to use this API through Python and get real-world distance between cities.

Almost all APIs require you to sign-up and obtain a *key*. The *key* is used to identify you and enforce usage limits so that you do not overwhelm the servers. We will obtain a key from OpenRouteServie so we can use their API

Visit [OpenRouteService Sign-up page](https://openrouteservice.org/dev/#/signup) and create an account. Once your account is activated, visit your Dashboard and request a token. Select *Standard* as the Token type and enter ``python_foundation`` as the Token name. Click *CREATE TOKEN*. Once created, copy the long string displayed under Key and enter below.

In [5]:
ORS_API_KEY = '5b3ce3597851110001cf6248b138a4a6c56f49e0a9251c75ad1de4cb'

We will use the OpenRouteServices's [Directions Service](https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/get). This service returns the driving, biking or walking directions between the given origin and destination points.

In [6]:
import requests

san_francisco = (37.7749, -122.4194)
new_york = (40.661, -73.944)

parameters = {
    'api_key': ORS_API_KEY,
    'start' : '{},{}'.format(san_francisco[1], san_francisco[0]),
    'end' : '{},{}'.format(new_york[1], new_york[0])
}

response = requests.get(
    'https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)

if response.status_code == 200:
    print('Request successful.')
    data = response.json()
else:
    print('Request failed.')


Request successful.


We can read the `response` in JSON format by calling `json()` method on it.

In [26]:
data = response.json()
print(data.__class__)
print(data.keys())
print(data['features'].__class__)
print(data['features'][0].__class__)
print(len(data['features']))
print(data['features'][0].__class__)
print(data['features'][0].keys())
print(data['features'][0]['properties'])

<class 'dict'>
dict_keys(['type', 'metadata', 'features', 'bbox'])
<class 'list'>
<class 'dict'>
1
<class 'dict'>
dict_keys(['bbox', 'type', 'properties', 'geometry'])
{'transfers': 0, 'fare': 0, 'segments': [{'distance': 4692230.9, 'duration': 166769.3, 'steps': [{'distance': 247.8, 'duration': 33.0, 'type': 11, 'instruction': 'Head northeast on Market Street', 'name': 'Market Street', 'way_points': [0, 5]}, {'distance': 191.5, 'duration': 24.6, 'type': 1, 'instruction': 'Turn right onto 10th Street', 'name': '10th Street', 'way_points': [5, 8]}, {'distance': 1917.1, 'duration': 188.6, 'type': 0, 'instruction': 'Turn left onto Mission Street', 'name': 'Mission Street', 'way_points': [8, 35]}, {'distance': 125.2, 'duration': 10.0, 'type': 1, 'instruction': 'Turn right onto New Montgomery Street', 'name': 'New Montgomery Street', 'way_points': [35, 37]}, {'distance': 88.5, 'duration': 21.3, 'type': 0, 'instruction': 'Turn left onto Natoma Street', 'name': 'Natoma Street', 'way_points': 

The response is a GeoJSON object representing the driving direction between the 2 points. The object is a feature collection with just 1 feature. We can access it using the index **0**. The feature's property contains `summary` information which has the data we need. 

In [27]:
summary = data['features'][0]['properties']['summary']
print(summary)

{'distance': 4692230.9, 'duration': 166769.3}


We can extract the `distance` and convert it to kilometers.

In [28]:
distance = summary['distance']
print(distance/1000)

4692.2309000000005


You can compare this distance to the straight-line distance and see the difference.

## Exercise 1

Replace the `ORS_API_KEY` with your own key in the code below. Change the cities with your chosen cities and run the cell to see the summary of driving directions. Extract the values for `distance` (meters) and `duration` (seconds). Convert and print the driving distance in km and driving time in minutes.

In [30]:
import requests

ORS_API_KEY = '5b3ce3597851110001cf6248b138a4a6c56f49e0a9251c75ad1de4cb'

boston = (42.361145, -71.057083)
san_diego = (32.715736, -117.161087)

parameters = {
    'api_key': ORS_API_KEY,
    'start' : '{},{}'.format(boston[1], boston[0]),
    'end' : '{},{}'.format(san_diego[1], san_diego[0])
}

response = requests.get(
    'https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)

if response.status_code == 200:
    print('Request successful.')
    data = response.json()
else:
    print('Request failed.')

data = response.json()

summary = data['features'][0]['properties']['summary']
print(summary)

print('{}: {}'.format('distance (km)', summary['distance']/1000))

Request successful.
{'distance': 4866087.0, 'duration': 177274.0}
distance: 4866.087


## API Rate Limiting

Many web APIs enforce *rate limiting* - allowing a limited number of requests over time. With computers it is easy to write a for loop, or have multiple programs send hundrends or thousands of queries per second. The server may not be configured to handle such volume. So the providers specify the limits on how many and how fast the queries can be sent. 

OpenRouteService lists several [API Restrictions](https://openrouteservice.org/plans/). The free plan allows for upto 40 direction requests/minute. 

There are many libraries available to implement various strategies for rate limiting. But we can use the built-in `time` module to implement a very simple rate limiting method.

### The `time` module

Python Standard Library has a `time` module that allows for time related operation. It contains a method `time.sleep()` which delays the execution of the program for the specified number of seconds.

In [31]:
import time
for x in range(10):
    print(x)
    time.sleep(1)

0
1
2
3
4
5
6
7
8
9


## Exercise 2

Below cell contains a dictionary with 3 destination cities and their coordinates. Write a `for` loop to iterate over the `destination_cities` disctionary and call `get_driving_distance()` function to print real driving distance between San Fransico and each city. Rate limit your queries by adding `time.sleep(2)` between successive function calls. Make sure to replace the `ORS_API_KEY` value with your own key.

In [35]:
import csv
import os
import requests
import time
ORS_API_KEY = '5b3ce3597851110001cf6248b138a4a6c56f49e0a9251c75ad1de4cb'

def get_driving_distance(source_coordinates, dest_coordinates):
    parameters = {
    'api_key': ORS_API_KEY,
    'start' : '{},{}'.format(source_coordinates[1], source_coordinates[0]),
    'end' : '{},{}'.format(dest_coordinates[1], dest_coordinates[0])
    }

    response = requests.get(
        'https://api.openrouteservice.org/v2/directions/driving-car', params=parameters)

    if response.status_code == 200:
        data = response.json()
        summary = data['features'][0]['properties']['summary']
        distance = summary['distance']
        return distance/1000
    else:
        print('Request failed.')
        return -9999

san_francisco = (37.7749, -122.4194)

destination_cities = {
    'Los Angeles': (34.0522, -118.2437),
    'Boston': (42.3601, -71.0589),
    'Atlanta': (33.7490, -84.3880)
}

for city, coords in destination_cities.items():
    print('{}:'.format(city), end=' ')
    distance = get_driving_distance(san_francisco, coords)
    print('{} [m]'.format(distance))
    time.sleep(2)

Los Angeles: 615.3069 [m]
Boston: 4986.5563 [m]
Atlanta: 3974.247 [m]


----

# Assignment
Your assignment is to geocode the addresses given below using [GeoPy](https://geopy.readthedocs.io/en/stable/). This assignment is designed to help you practice your coding skills learnt in the course so far.

# Part 1
You have been given a list containing 5 tuples of place names along with their address. You need to use the [Nominatim](https://geopy.readthedocs.io/en/stable/#module-geopy.geocoders) geocoder and obtain the latitude and longitude of each address.

### The expected output should be as follows
[<br>('Norman Thomas HS (ECF)', 40.7462177, -73.9809816),<br>
 ('Midtown East Campus', 40.65132465, -73.92421646290632),<br>
 ('Louis D. Brandeis HS', 40.7857432, -73.9742029),<br>
 ('Martin Luther King, Jr. HS', 40.7747751, -73.9853689),<br>
 ('P.S. 48', 40.8532731, -73.9338592)<br>]<br>


In [1]:
from geopy.geocoders import Nominatim

# List of Hurricane Evacuation Centers in New York City with Addresses
# Each item is a tuple with the name of the center and its address
locations = [
    ('Norman Thomas HS (ECF)', '111 E 33rd St, NYC, New York'),
    ('Midtown East Campus', '233 E 56th St, NYC, New York'),
    ('Louis D. Brandeis HS', '145 W 84th St, NYC, New York'),
    ('Martin Luther King, Jr. HS', '122 Amsterdam Avenue, NYC, New York'),
    ('P.S. 48', '4360 Broadway, NYC, New York')
]

In [2]:
# Specify a custom user agent
geolocator = Nominatim(user_agent="jon-application")

# Perform geocoding
location = geolocator.geocode("175 5th Avenue NYC")

url = location.raw.get("geotext")
print(url)

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=175+5th+Avenue+NYC&format=json&limit=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: certificate has expired (_ssl.c:1007)')))

In [3]:
import geopy
print(geopy.__version__)

2.4.0
